# RecentPopularity Algorithm on Test Dataset

In [ ]:
k = 2
from streamsightv2.datasets import TestDataset
from streamsightv2.settings import SlidingWindowSetting
dataset = TestDataset()
data = dataset.load()
setting_window = SlidingWindowSetting(
    4,
    3,
    0,
    k
)
setting_window.split(data)

INFO - streamsight package loaded.
WARNING - /Users/joshua/projects/fyp/streamsightv2/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

DEBUG - TestDataset being initialized with 'data' as the base path.
DEBUG - TestDataset is initialized.
INFO - TestDataset is loading dataset...
DEBUG - TestDataset applying filters set.
DEBUG - 	interactions before preprocess: 13
DEBUG - 	items before preprocess: 3
DEBUG - 	users before preprocess: 5
DEBUG - 	interactions after preprocess: 13
DEBUG - 	items after preprocess: 3
DEBUG - 	users after preprocess: 5
INFO - TestDataset dataset loaded - Took 0.0277s
DEBUG - Splitting data...
DEBUG - Performing lt(t, 2147483647)
DEBUG - Performing lt(t, 4)
DEBUG - Performing ge(t, 4)
DEBUG - TimestampSplitter(t=4,t_lower=None,t_upper=None) has complete split


  0%|          | 0/2 [00:00<?, ?it/s]

DEBUG - NPastInteractionTimestampSplitter(t=4,t_lower=None,t_upper=3,n_seq_data=0,include_all_past_data=False) - Updating split point to t=4
DEBUG - Performing lt(t, 7)
DEBUG - Performing ge(t, 4)
DEBUG - Performing get_user_n_last_interaction comparison
DEBUG - NPastInteractionTimestampSplitter(t=4,t_lower=None,t_upper=3,n_seq_data=0,include_all_past_data=False) has complete split
INFO - Split at time 4 resulted in empty unlabelled testing samples.
DEBUG - NPastInteractionTimestampSplitter(t=4,t_lower=None,t_upper=3,n_seq_data=0,include_all_past_data=False) - Updating split point to t=7
DEBUG - Performing lt(t, 10)
DEBUG - Performing ge(t, 7)
DEBUG - Performing get_user_n_last_interaction comparison
DEBUG - NPastInteractionTimestampSplitter(t=7,t_lower=None,t_upper=3,n_seq_data=0,include_all_past_data=False) has complete split
INFO - Split at time 7 resulted in empty unlabelled testing samples.
DEBUG - NPastInteractionTimestampSplitter(t=7,t_lower=None,t_upper=3,n_seq_data=0,include_a

3it [00:00, 62.45it/s]               

INFO - Finished split with window size 3 seconds. Number of splits: 3 in total.
INFO - SlidingWindowSetting data split - Took 0.0801s
DEBUG - Checking split attribute and sizes.
DEBUG - Checking split attributes.
DEBUG - Split attributes are set.
DEBUG - Checking size of split sets.
DEBUG - Size of split sets are checked.
INFO - SlidingWindowSetting data split complete.


In [ ]:
from streamsightv2.evaluators import EvaluatorStreamerBuilder

builder = EvaluatorStreamerBuilder()
builder.add_setting(setting_window)
builder.set_metric_K(k)
# builder.add_metric("PrecisionK")
builder.add_metric("RecallK")
evaluator = builder.build()

In [ ]:
from streamsightv2.algorithms import RecentPopularity
# from recpack.algorithms import Popularity
external_model = RecentPopularity(K=2)
# external_model = ItemKNNIncremental(K=10)
external_model_id = evaluator.register_algorithm(algorithm_name="my own algo")
print(external_model_id)

INFO - Registering algorithm name my own algo with ID: bdd640fb-0667-4ad1-9c80-317fa3b1799d
DEBUG - Algorithm bdd640fb-0667-4ad1-9c80-317fa3b1799d registered
bdd640fb-0667-4ad1-9c80-317fa3b1799d


In [4]:
evaluator.start_stream()

DEBUG - Resetting data generators.
DEBUG - Data generators are reset.
DEBUG - Preparing evaluator for streaming
DEBUG - (user x item) shape defined is (3, 3)
DEBUG - Shape of dataframe stored in matrix was (4, 5) before masking
DEBUG - Shape of dataframe stored in matrix is now (4, 5) after masking
DEBUG - Final (user x item) shape defined is (3, 3)
DEBUG - Caching evaluation data for step 1
DEBUG - (user x item) shape defined is (3, 3)
DEBUG - Shape of dataframe stored in matrix was (3, 5) before masking
DEBUG - Shape of dataframe stored in matrix is now (3, 5) after masking
DEBUG - Final (user x item) shape defined is (3, 3)
DEBUG - (user x item) shape defined is (3, 3)
DEBUG - Shape of dataframe stored in matrix was (3, 5) before masking
DEBUG - Shape of dataframe stored in matrix is now (3, 5) after masking
DEBUG - Final (user x item) shape defined is (4, 3)
DEBUG - Data cached for step 1 complete


# Iteration 1

In [5]:
data = evaluator.get_data(external_model_id)
external_model.fit(data)
data

DEBUG - Getting data for algorithm bdd640fb-0667-4ad1-9c80-317fa3b1799d
X_transformed:  [[1 0 1]
 [1 0 0]
 [0 1 0]]
Sorted scores recentpop:  [1.  0.  0.5]
DEBUG - Fitting RecentPopularity complete - Took 0.00493s


   interactionid  uid  iid  ts  rating
0              0    0    0   0       1
1              1    1    0   1       2
2              2    2    1   2       3
3              3    0    2   3       4

In [6]:
ul_data = evaluator.get_unlabeled_data(external_model_id)
prediction = external_model.predict(data, ul_data)
ul_data

DEBUG - Getting unlabeled data for algorithm bdd640fb-0667-4ad1-9c80-317fa3b1799d
DEBUG - Performing items_in comparison
Users:  [1, 3]
Known item id:  3
Max user id:  4
Intended shape:  (4, 3)
X_pred:  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
X_pred after:  [[0.  0.  0. ]
 [1.  0.  0.5]
 [0.  0.  0. ]
 [1.  0.  0.5]]


   interactionid  uid  iid  ts  rating
4              4    1   -1   4       5
5              5    1   -1   5       1
6              6    3   -1   6       2

In [7]:
print(prediction)
print(prediction.toarray())
evaluator.submit_prediction(external_model_id, prediction)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 4 stored elements and shape (4, 3)>
  Coords	Values
  (1, 0)	1.0
  (1, 2)	0.5
  (3, 0)	1.0
  (3, 2)	0.5
[[0.  0.  0. ]
 [1.  0.  0.5]
 [0.  0.  0. ]
 [1.  0.  0.5]]
DEBUG - Submitting prediction for algorithm bdd640fb-0667-4ad1-9c80-317fa3b1799d
DEBUG - Precision compute started - RecallK_2
DEBUG - Number of users: 2
DEBUG - Number of ground truth interactions: 3
DEBUG - Recall compute complete - RecallK_2
DEBUG - Metric RecallK(timestamp_limit=4,K=2) created for algorithm my own algo_bdd640fb-0667-4ad1-9c80-317fa3b1799d
DEBUG - Prediction evaluated for algorithm bdd640fb-0667-4ad1-9c80-317fa3b1799d complete


# Iteration 2

In [8]:
data = evaluator.get_data(external_model_id)
external_model.fit(data)
data

DEBUG - Getting data for algorithm bdd640fb-0667-4ad1-9c80-317fa3b1799d
DEBUG - (user x item) shape defined is (4, 3)
DEBUG - Shape of dataframe stored in matrix was (3, 5) before masking
DEBUG - Shape of dataframe stored in matrix is now (3, 5) after masking
DEBUG - Final (user x item) shape defined is (4, 3)
DEBUG - Caching evaluation data for step 2
DEBUG - (user x item) shape defined is (4, 3)
DEBUG - Shape of dataframe stored in matrix was (4, 5) before masking
DEBUG - Shape of dataframe stored in matrix is now (4, 5) after masking
DEBUG - Final (user x item) shape defined is (4, 3)
DEBUG - (user x item) shape defined is (4, 3)
DEBUG - Shape of dataframe stored in matrix was (4, 5) before masking
DEBUG - Shape of dataframe stored in matrix is now (4, 5) after masking
DEBUG - Final (user x item) shape defined is (5, 3)
DEBUG - Data cached for step 2 complete
X_transformed:  [[0 0 0]
 [0 1 1]
 [0 0 0]
 [0 1 0]]
Sorted scores recentpop:  [0.  1.  0.5]
DEBUG - Fitting RecentPopularity

   interactionid  uid  iid  ts  rating
4              4    1    1   4       5
5              5    1    2   5       1
6              6    3    1   6       2

In [9]:
ul_data = evaluator.get_unlabeled_data(external_model_id)
prediction = external_model.predict(data, ul_data)
ul_data

DEBUG - Getting unlabeled data for algorithm bdd640fb-0667-4ad1-9c80-317fa3b1799d
DEBUG - Performing items_in comparison
Users:  [2, 3, 4]
Known item id:  3
Max user id:  5
Intended shape:  (5, 3)
X_pred:  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
X_pred after:  [[0.  0.  0. ]
 [0.  0.  0. ]
 [0.  1.  0.5]
 [0.  1.  0.5]
 [0.  1.  0.5]]


    interactionid  uid  iid  ts  rating
7               7    2   -1   7       3
8               8    2   -1   8       4
9               9    3   -1   9       5
10             10    4   -1   9       1

In [10]:
print(prediction)
print(prediction.toarray())
evaluator.submit_prediction(external_model_id, prediction)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 6 stored elements and shape (5, 3)>
  Coords	Values
  (2, 1)	1.0
  (2, 2)	0.5
  (3, 1)	1.0
  (3, 2)	0.5
  (4, 1)	1.0
  (4, 2)	0.5
[[0.  0.  0. ]
 [0.  0.  0. ]
 [0.  1.  0.5]
 [0.  1.  0.5]
 [0.  1.  0.5]]
DEBUG - Submitting prediction for algorithm bdd640fb-0667-4ad1-9c80-317fa3b1799d
DEBUG - Precision compute started - RecallK_2
DEBUG - Number of users: 3
DEBUG - Number of ground truth interactions: 4
DEBUG - Recall compute complete - RecallK_2
DEBUG - Metric RecallK(timestamp_limit=7,K=2) created for algorithm my own algo_bdd640fb-0667-4ad1-9c80-317fa3b1799d
DEBUG - Prediction evaluated for algorithm bdd640fb-0667-4ad1-9c80-317fa3b1799d complete


# Iteration 3

In [11]:
data = evaluator.get_data(external_model_id)
external_model.fit(data)
data

DEBUG - Getting data for algorithm bdd640fb-0667-4ad1-9c80-317fa3b1799d
DEBUG - (user x item) shape defined is (5, 3)
DEBUG - Shape of dataframe stored in matrix was (4, 5) before masking
DEBUG - Shape of dataframe stored in matrix is now (4, 5) after masking
DEBUG - Final (user x item) shape defined is (5, 3)
DEBUG - Caching evaluation data for step 3
DEBUG - (user x item) shape defined is (5, 3)
DEBUG - Shape of dataframe stored in matrix was (2, 5) before masking
DEBUG - Shape of dataframe stored in matrix is now (2, 5) after masking
DEBUG - Final (user x item) shape defined is (5, 3)
DEBUG - (user x item) shape defined is (5, 3)
DEBUG - Shape of dataframe stored in matrix was (2, 5) before masking
DEBUG - Shape of dataframe stored in matrix is now (2, 5) after masking
DEBUG - Final (user x item) shape defined is (5, 3)
DEBUG - Data cached for step 3 complete
X_transformed:  [[0 0 0]
 [0 0 0]
 [1 0 1]
 [0 0 1]
 [1 0 0]]
Sorted scores recentpop:  [1. 0. 1.]
DEBUG - Fitting RecentPopu

    interactionid  uid  iid  ts  rating
7               7    2    0   7       3
8               8    2    2   8       4
9               9    3    2   9       5
10             10    4    0   9       1

In [12]:
ul_data = evaluator.get_unlabeled_data(external_model_id)
prediction = external_model.predict(data, ul_data)
ul_data

DEBUG - Getting unlabeled data for algorithm bdd640fb-0667-4ad1-9c80-317fa3b1799d
DEBUG - Performing items_in comparison
Users:  [4]
Known item id:  3
Max user id:  5
Intended shape:  (5, 3)
X_pred:  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
X_pred after:  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [1. 0. 1.]]


    interactionid  uid  iid  ts  rating
11             11    4   -1  10       2
12             12    4   -1  10       3

In [13]:
print(prediction)
print(prediction.toarray())
evaluator.submit_prediction(external_model_id, prediction)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 2 stored elements and shape (5, 3)>
  Coords	Values
  (4, 0)	1.0
  (4, 2)	1.0
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [1. 0. 1.]]
DEBUG - Submitting prediction for algorithm bdd640fb-0667-4ad1-9c80-317fa3b1799d
DEBUG - Precision compute started - RecallK_2
DEBUG - Number of users: 1
DEBUG - Number of ground truth interactions: 2
DEBUG - Recall compute complete - RecallK_2
DEBUG - Metric RecallK(timestamp_limit=10,K=2) created for algorithm my own algo_bdd640fb-0667-4ad1-9c80-317fa3b1799d
DEBUG - Prediction evaluated for algorithm bdd640fb-0667-4ad1-9c80-317fa3b1799d complete
INFO - Finished streaming
WARNING - /Users/joshua/projects/fyp/streamsightv2/venv/lib/python3.12/site-packages/streamsight/evaluators/evaluator_stream.py:379: AlgorithmStatusWarning: Algorithm:bdd640fb-0667-4ad1-9c80-317fa3b1799d current status is READY. Algorithm has completed stream evaluation. No more data release available.
  warn(AlgorithmS

# evaluate metrics

In [14]:
evaluator.get_all_algorithm_status()

{'my own algo_bdd640fb-0667-4ad1-9c80-317fa3b1799d': <AlgorithmStateEnum.COMPLETED: 'COMPLETED'>}

In [15]:
evaluator.metric_results("macro")

,,macro_score,num_window
Algorithm,Metric,,
my own algo_bdd640fb-0667-4ad1-9c80-317fa3b1799d,RecallK_2,0.416667,3


In [16]:
evaluator.metric_results("micro")


,,micro_score,num_user
Algorithm,Metric,,
my own algo_bdd640fb-0667-4ad1-9c80-317fa3b1799d,RecallK_2,0.416667,6


In [18]:
evaluator.metric_results("user")

user_id  \
Algorithm                                        Timestamp Metric              
my own algo_bdd640fb-0667-4ad1-9c80-317fa3b1799d t=4       RecallK_2       1   
                                                           RecallK_2       3   
                                                 t=7       RecallK_2       2   
                                                           RecallK_2       3   
                                                           RecallK_2       4   
                                                 t=10      RecallK_2       4   

                                                                     score  
Algorithm                                        Timestamp Metric           
my own algo_bdd640fb-0667-4ad1-9c80-317fa3b1799d t=4       RecallK_2   0.5  
                                                           RecallK_2   0.0  
                                                 t=7       RecallK_2   0.5  
                                                           RecallK_2   1.0  
                                                           RecallK_2   0.0  
                                                 t=10      RecallK_2   0.5

In [19]:
evaluator.metric_results("window")

window_score  \
Algorithm                                        Timestamp Metric                    
my own algo_bdd640fb-0667-4ad1-9c80-317fa3b1799d t=4       RecallK_2          0.25   
                                                 t=7       RecallK_2          0.50   
                                                 t=10      RecallK_2          0.50   

                                                                      num_user  
Algorithm                                        Timestamp Metric               
my own algo_bdd640fb-0667-4ad1-9c80-317fa3b1799d t=4       RecallK_2         2  
                                                 t=7       RecallK_2         3  
                                                 t=10      RecallK_2         1